# Start a chat with LLM

The `Session` object is at the core of lionagi.

A session is an interface to
- manages and logs various messages (system, user, assistant) in a conversation, 
- get AI model inferencing with `Services`
- enable effortless multi-round exchange between many participants. 

A Session object contains one or more `branch`, by default, every session has a `'main'` branch， `session.branches['main']`

- `branch.messages` is a `pd.DataFrame`

lionagi has some default config, you can check them under lionagi.integrations.config

In [1]:
import lionagi as li
from IPython.display import Markdown  # for better print

In [2]:
from lionagi.integrations.config import oai_schema

oai_schema["chat/completions"]["config"]

{'model': 'gpt-4o',
 'frequency_penalty': 0,
 'max_tokens': None,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'text'},
 'seed': None,
 'stop': None,
 'stream': False,
 'temperature': 0.1,
 'top_p': 1,
 'tools': None,
 'tool_choice': 'none',
 'user': None,
 'logprobs': False,
 'top_logprobs': None}

In [3]:
# create some messages
sys_comedian = "As a comedian, you are sarcastically funny"
instruct1 = (
    "very short joke: a blue whale and a big shark meet at the bar and start dancing"
)

In [4]:
# create a llm conversation with OpenAI gpt-4 (default)
comedian1 = li.Branch(system=sys_comedian)
joke1 = await comedian1.chat(instruction=instruct1, max_tokens=50)

In [5]:
Markdown(joke1)

And the bartender says, "Great, now I have to mop up the ocean!"

In [6]:
comedian1.imodel.to_dict()

{'ln_id': '99277cfab27ed2b0ba872dd3eaeff55f',
 'timestamp': '2024-05-28T16:41:34.177124',
 'provider': 'OpenAI',
 'api_key': 'sk-r*******************************************SgD6',
 'endpoint': 'chat/completions',
 'token_encoding_name': 'cl100k_base',
 'model': 'gpt-4o',
 'frequency_penalty': 0,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'text'},
 'stream': False,
 'temperature': 0.1,
 'top_p': 1,
 'tool_choice': 'none',
 'logprobs': False,
 'model_costs': (5, 15)}

You can change a LLM provider by choosing a pre-configured service, or create your own

In [7]:
imodel = li.iModel(
    model="mistralai/mistral-7b-instruct",
    provider="OpenRouter",
    api_key_schema="OPENROUTER_API_KEY",
    costs=(0.07, 0.07),
)

comedian2 = li.Branch(system=sys_comedian, imodel=imodel)
joke2 = await comedian2.chat(instruction=instruct1, max_tokens=50)

In [8]:
imodel.to_dict()

{'ln_id': 'aae7bd3849b33ca6eceb81212da66bb1',
 'timestamp': '2024-05-28T16:41:35.207140',
 'provider': 'OpenRouter',
 'api_key': 'sk-o*****************************************************************4cca',
 'endpoint': 'chat/completions',
 'token_encoding_name': 'cl100k_base',
 'model': 'mistralai/mistral-7b-instruct',
 'frequency_penalty': 0,
 'presence_penalty': 0,
 'response_format': {'type': 'text'},
 'stream': False,
 'temperature': 0.1,
 'top_p': 1,
 'tool_choice': 'none',
 'logprobs': False,
 'model_costs': (0.07, 0.07)}

In [9]:
Markdown(joke2)

The bartender asks, "What is this, some kind of a whale-shark tango?" The blue whale replies, "No, just two sea creatures trying to avoid the dance of death." *audience groans

In [10]:
comedian2.to_df()

,ln_id,message_type,timestamp,role,content,metadata,sender,recipient
0,330a5e7aea6095a79a2ec20063468f7e,System,2024-05-28T16:41:35.207426,system,"{'system_info': 'As a comedian, you are sarcas...",{'last_updated': {'recipient': '2024-05-28T16:...,system,199fd7fe872f7e9e1c945da66cd84bc6
1,5a3198fc7485594450490320d9bc4c43,Instruction,2024-05-28T16:41:35.207940,user,{'instruction': 'very short joke: a blue whale...,{'last_updated': {'sender': '2024-05-28T16:41:...,user,199fd7fe872f7e9e1c945da66cd84bc6
2,96c9e1ea57da7e6b7a0e67ea8a8fe24f,AssistantResponse,2024-05-28T16:41:36.941473,assistant,"{'assistant_response': 'The bartender asks, ""W...",{'last_updated': {'sender': '2024-05-28T16:41:...,199fd7fe872f7e9e1c945da66cd84bc6,user


In [11]:
comedian2.messages[-1].metadata

{'last_updated': {'sender': '2024-05-28T16:41:36.941606',
  'recipient': '2024-05-28T16:41:36.941621'},
 'extra': {'id': 'gen-mlCtEFnwIRug8NK1ySOlRx4egwTm',
  'model': 'mistralai/mistral-7b-instruct',
  'object': 'chat.completion',
  'created': 1716914495,
  'usage': {'prompt_tokens': 38,
   'completion_tokens': 50,
   'total_tokens': 88,
   'expense': 6.16e-06},
  'index': 0,
  'finish_reason': 'length'}}